In [21]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
import json

browser = Browser('chrome')

# # Chrome options and preferences
# chrome_options = Options()
# chrome_options.add_argument("--blink-settings=imagesEnabled=false")

# # Chrome with options
# browser = Browser('chrome', options=chrome_options)


In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
cookie = browser.find_by_id('btn-cookie-allow').click()
# cookie.click()

In [4]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(1)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

0
1
2


In [5]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)
        

In [6]:
len(links)

167

In [7]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []


max_iteration2 = 5
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()
    
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        regular_price.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        sale_price.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        name.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        onsale.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        reviews.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        size.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        description.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
#     try:
#         moretaste = soup.find('div', class_ = 'foodParings pip-info')
#         all_taste = moretaste.find_all('li')
#         taste = {}

#         for taste_record in all_taste:
#             taste_value1 = taste_record.find('div', class_ = 'value').text
#             taste_name = taste_record.find('div', class_ = 'label').text
#             taste_value = taste_value1.get('aria-label')
#             taste[taste_name] = taste_value
#         taste_list.append(taste)
#     except AttributeError:
#         taste_list.append('N/A')
    
    print(iteration2)
    iteration2 += 1



0
1
2
3
4


In [19]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list
}

# # Add the details dictionary to the data dictionary
# data['Details'] = details_list
# # data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# # taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# # df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# # df2.drop('Taste', axis = 1, inplace = True)

# # Display the dataframe Test
df.head(20)

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal,Release Date
0,Ruffino Prosecco Rose DOC,$20.00,$18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN
1,Josh Cellars Pinot Grigio,$19.95,$17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN
2,Pelee Island Best in Show Pinot Grigio,$11.95,$9.95,3.0,(2),750 ml bottle,No wonder this winery is the best in show - it...,White Wine,12.5%,Canada,Pelee Island Winery,10 g/L,Pinot Grigio,NaN
3,Peller Family Vineyards Pinot Grigio,$10.95,$9.95,5.0,(3),750 ml bottle,"Easy drinking, refreshing white wine, sourced ...",White Wine,12%,Canada,Peller Family Vineyards,6 g/L,Pinot Grigio,NaN
4,Santa Margherita Brut Valdobbiadene Prosecco ...,$20.95,$18.95,4.8,(12),750 ml bottle,We've heard this Prosecco is a celebrity favou...,White Sparkling Wine,11.5%,"Veneto, Italy",Santa Margherita,13 g/L,Sparkling - Dry,"April 1, 2022"


In [20]:
df['Lat'] = ""
df['Lon'] = ""   
df.head()

,Name,Regular Price,Sale Price,Rating,Reviews,Size,Description,Category,Alcohol/Vol,Made In,By,Sugar Content,Varietal,Release Date,Lat,Lon
0,Ruffino Prosecco Rose DOC,$20.00,$18.00,5.0,(2),750 ml bottle,Ruffino has been at the forefront of Italian w...,Rosé & Red Sparkling Wine,11%,"Friuli, Italy",Ruffino Srl Wines,21 g/L,Sparkling,NaN,,
1,Josh Cellars Pinot Grigio,$19.95,$17.95,4.5,(17),750 ml bottle,Josh Cellars is a renowned brand created to ce...,White Wine,12.5%,"California, United States",Josh Cellars,3 g/L,Pinot Grigio,NaN,,
2,Pelee Island Best in Show Pinot Grigio,$11.95,$9.95,3.0,(2),750 ml bottle,No wonder this winery is the best in show - it...,White Wine,12.5%,Canada,Pelee Island Winery,10 g/L,Pinot Grigio,NaN,,
3,Peller Family Vineyards Pinot Grigio,$10.95,$9.95,5.0,(3),750 ml bottle,"Easy drinking, refreshing white wine, sourced ...",White Wine,12%,Canada,Peller Family Vineyards,6 g/L,Pinot Grigio,NaN,,
4,Santa Margherita Brut Valdobbiadene Prosecco ...,$20.95,$18.95,4.8,(12),750 ml bottle,We've heard this Prosecco is a celebrity favou...,White Sparkling Wine,11.5%,"Veneto, Italy",Santa Margherita,13 g/L,Sparkling - Dry,"April 1, 2022",,


In [22]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

NameError: name 'geoapify_key' is not defined